<a href="https://colab.research.google.com/github/MAfarrag/HAPI/blob/master/Examples/03Jiboa-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jiboa Case Study

This code is prepared to Run the distributed model for jiboa rover in El Salvador
wher the catchment is consisted os a ustream lake and a volcanic area
-   you have to make the root directory to the examples folder to enable the code
    from reading input files

In [ ]:
# install libraries
!pip install Numpy
!pip install pandas
!pip install pandas
!pip install gdal
!pip install fiona
!pip install shapely
!pip install geopandas
!pip install shapely
!pip install git+https://github.com/MAfarrag/HAPI.git

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Download Data
For the data of this case study you have to download this folder [Jiboa Data](https://drive.google.com/drive/folders/1uyd9mH8pHHUKK9l3bc7QXlsu4EPwy3Mr?usp=sharing) from Google Drive and set it as the working directory instead of the Path defined in the next cell

## Import modules

In [3]:
import os
Path = "/content/drive/My Drive/"

os.chdir(Path)

#%library
import gdal
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# HAPI modules
from Hapi.run import RunHAPIwithLake
import Hapi.hbv as HBV
import Hapi.performancecriteria as Pf
import Hapi.raster as Raster

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# the beginning of the simulation of the calibration data
start = dt.datetime(2012,6,14,19,00,00)
end = dt.datetime(2014,11,17,00,00,00)
calib_end = dt.datetime(2013,12,23,00,00,00)

# paths to the meteorological data
PrecPath = prec_path = "inputs/Hapi/meteodata/4000/calib/prec_clipped"
Evap_Path = evap_path = "inputs/Hapi/meteodata/4000/calib/evap_clipped"
TempPath = temp_path = "inputs/Hapi/meteodata/4000/calib/temp_clipped"

#DemPath = path+"GIS/4000/dem4000.tif"
FlowAccPath = "inputs/Hapi/GIS/4000_matched/acc4000.tif"
FlowDPath = "inputs/Hapi/GIS/4000_matched/fd4000.tif"
ParPath = "inputs/Hapi/meteodata/4000/parameters/"
#ParPath = "inputs/Hapi/meteodata/4000/"+"parameters.txt"
Paths=[PrecPath, Evap_Path, TempPath, FlowAccPath, FlowDPath, ]

In [ ]:

#p2=[24, 1530]
#init_st=[0,5,5,5,0]
init_st = np.loadtxt("inputs/Hapi/meteodata/Initia-jiboa.txt", usecols=0).tolist()
snow = 0


# lake meteorological data
ind = pd.date_range(start, end, freq = "H" )
lakedata = pd.read_csv("inputs/Hapi/meteodata/lakedata.csv", index_col = 0)
lakedata.index = ind
lakeCalib = lakedata.loc[start:calib_end]
lakeValid = lakedata.loc[calib_end:end]
# convert the dataframe into array
lakeCalibArray = lakeCalib.values
# take only the plake, et, t and tm columns and exclude the last column
lakeCalibArray = lakeCalibArray[:,0:-1]

# where the lake discharges its flow (give the indices of the cell)
lakecell = [2,1]    # 4km
#lakecell = [4,2]    # 2km
#lakecell = [10,4]    # 1km
#lakecell = [19,10]    # 500m

LakeParameters = np.loadtxt("inputs/Hapi/meteodata/4000/Lakeparameters.txt").tolist()
StageDischargeCurve = np.loadtxt("inputs/Hapi/meteodata/curve.txt")
p2 = [1, 227.31, 133.98, 70.64]
Lake_init_st = np.loadtxt("inputs/Hapi/meteodata/Initia-lake.txt", usecols=0).tolist()

# Run the model

In [12]:
os.getcwd()

'/content/drive/My Drive'

In [13]:
os.listdir()

['Hapi', 'Jiboa.mxd', 'web application', 'processing', 'inputs', 'results.zip']

In [ ]:
Sim =pd.DataFrame(index = lakeCalib.index)
st, Sim['Q'], q_uz_routed, q_lz_trans = RunHAPIwithLake(HBV, Paths, ParPath, p2, init_st,
                                                     snow, lakeCalibArray, StageDischargeCurve,
                                                     LakeParameters, lakecell,Lake_init_st)

### Evaluate model performance

In [ ]:
WS = {}
WS['type'] = 1
WS['N'] = 3
ModelMetrics=dict()
ModelMetrics['CalibErrorHf']=Pf.RMSEHF(lakeCalib['Q'],Sim['Q'],WS['type'],WS['N'],0.75)
ModelMetrics['CalibErrorLf']=Pf.RMSELF(lakeCalib['Q'],Sim['Q'],WS['type'],WS['N'],0.75)
ModelMetrics['CalibNSEHf']=Pf.NSE(lakeCalib['Q'],Sim['Q'])
ModelMetrics['CalibNSELf']=Pf.NSE(np.log(lakeCalib['Q']),np.log(Sim['Q']))
ModelMetrics['CalibRMSE']=Pf.RMSE(lakeCalib['Q'],Sim['Q'])
ModelMetrics['CalibKGE']=Pf.KGE(lakeCalib['Q'],Sim['Q'])
ModelMetrics['CalibWB']=Pf.WB(lakeCalib['Q'],Sim['Q'])
print("RMSE(HF) = " + str(round(ModelMetrics['CalibErrorHf'],2)))
print("RMSE(LF) = " + str(round(ModelMetrics['CalibErrorLf'],2)))
print("RMSE = " + str(round(ModelMetrics['CalibRMSE'],2)))
print("NSE(HF) = " + str(round(ModelMetrics['CalibNSEHf'],2)))
print("NSE(LF) = " + str(round(ModelMetrics['CalibNSELf'],2)))
print("KGE = " + str(round(ModelMetrics['CalibKGE'],2)))
print("WB = " + str(round(ModelMetrics['CalibWB'],2)))

In [ ]:
plt.figure(50,figsize=(15,8))
Sim.Q.plot(color=[(0,0.3,0.7)],linewidth=2.5,label="Simulated data", zorder = 10)
ax1=lakeCalib['Q'].plot(color='#DC143C',linewidth=2.8,label='Observed data')
ax1.annotate("Model performance" ,xy=('2012-12-01 00:00:00',20),fontsize=15)
ax1.annotate("RMSE = " + str(round(ModelMetrics['CalibRMSE'],3)),xy=('2012-12-01 00:00:00',20-1.5),fontsize=15)
ax1.annotate("NSE = " + str(round(ModelMetrics['CalibNSEHf'],2)),xy=('2012-12-01 00:00:00',20-3),fontsize=15)
plt.legend()

# Store the result into rasters

In [ ]:
# create list of names
src=gdal.Open(FlowAccPath)

index=pd.date_range(start,calib_end,freq="1H")

resultspath="results/upper_zone_discharge/4000/"
names=[resultspath+str(i)[:-6] for i in index]
names=[i.replace("-","_") for i in names]
names=[i.replace(" ","_") for i in names]
names=[i+".tif" for i in names]


Raster.RastersLike(src,q_uz_routed[:,:,:-1],names)
